In [1]:
import pandas as pd
import os

In [2]:
script_directory = os.path.dirname(os.path.abspath(__file__)) if '__file__' in locals() else os.getcwd()
relative_folder_path = "data"
folder_path = os.path.join(script_directory.replace('\code', ''), relative_folder_path)
file = os.listdir(folder_path)
path = f"{folder_path}\{file[0]}"
df_root = pd.read_excel(path)

In [22]:
df__lession  = df_root[df_root['name']=='Lesion Measurement']
df__lession = df__lession[['Visit','Participante', 'Campo', 'Valor']]
df__lession = df__lession[df__lession['Visit']=='Screening Visit']
df__lession = df__lession[df__lession['Campo']=='Anatomical Location']
df__lession = df__lession[['Visit','Participante','Valor']]
df__lession = df__lession.rename(columns={'Participante':'Subject', 'Valor':'lesion_measurement'})
df__lession['Count'] = len(df__lession['lesion_measurement'])
df__lession = df__lession[['Visit','Subject','Count']]
# df_with_counts = df__lession.merge(subject_counts, left_on='Subject', right_on='Subject')
# df_with_counts = df_with_counts[['Visit','Subject','Count']]

In [23]:
df__lession

,Visit,Subject,Count
300,Screening Visit,011001,4
309,Screening Visit,011001,4
1816,Screening Visit,S002,4
1825,Screening Visit,S003,4


In [4]:
df_root.activityState.unique()

array(['DATA_VERIFIED', 'DATA_ENTRY_COMPLETE', 'DATA_ENTRY_INCOMPLETE'],
      dtype=object)

In [4]:
df_root.Participante.unique()

array([11001], dtype=int64)

In [5]:
df_root.name.unique()

array(['Date of visit', 'Informed Consent', 'Demographics',
       'History of cutaneous leishmaniasis', 'Covid 19 testing',
       'Eligibility',
       'Medical Or Surgical History (other than Leishmaniasis)',
       'Vein assessment', 'Urinary Drug Screen', 'Alcohol Screen',
       'Clinical Laboratory - Test Hematology',
       'Clinical Laboratory Test - Clinical Chemistry',
       'Clinical Laboratory Test - Coagulation',
       'Immunoassay (Thyroid Stimulating Hormone)', 'Urinalysis',
       'Urine Microscopic Examination', 'Virology', 'Lesion Measurement',
       'Physical Examination', 'Vital Signs', '12-Lead ECG',
       'Clinical Laboratory Test - Clinical Chemistry D-1',
       'Interleukin-6', 'Titration Of Auto-Antibodies',
       'Injection Site Examination', 'PBMC Isolate',
       'Pharmacokinetic Blood Sampling (PK)',
       'Pharmacodynamic Blood Sampling (PD) -Cytokines/Chemokines',
       'mRNA Markers'], dtype=object)

In [5]:
pru = df_root[df_root['name']== '12-Lead ECG']

In [37]:
df_contraception = df_root[df_root['name']== 'Prior And Concomitant Medications']
df_contraception = df_contraception[['Visit','Participante', 'Campo', 'Valor', 'Instancia']]
df_contraception = df_contraception.sort_values(by=['FormFieldInstance Id'])
df_contraception = df_contraception.reset_index(drop=True)
date_indices = df_contraception.index[df_contraception['Campo'] == 'Concomitant Medication ID'].tolist()
subdatasets = [df_contraception.iloc[start:end] for start, end in zip(date_indices, date_indices[1:] + [None])]
df_to_join = pd.DataFrame()
for sub in subdatasets:
    #if sub['Valor'].tolist()  in [5, 5.0, '5', '5.0', 9]:
    if len([float(x) for x in sub['Valor'].tolist() if x in [5, 5.0, '5', '5.0', 9]]) != 0:
        df_to_join = sub[(sub['Campo'] == 'Indication Category') | (sub['Campo'] == 'Start date')]
        sujeto = df_to_join.Participante.unique()
        df_to_join = df_to_join[['Campo', 'Valor']].T
        new_columns = df_to_join.iloc[0]
        df_to_join = df_to_join[1:].set_axis(new_columns, axis=1)
        df_to_join['Subject'] = sujeto
        break
df_to_join = df_to_join.rename(columns={'Participante':'Subject', 'Start date':'start_date_combined_hormonal'})

KeyError: 'FormFieldInstance Id'

In [36]:
df_to_join

Campo,Birth Month,Birth Year,Subject
Valor,5,2000,S003


In [7]:
pru.Campo.unique()

array(['Was the ECG performed?', 'Date of ECG Performed',
       'Undefined, HR (bpm)', 'Undefined, RR (msec)',
       'Undefined, PR (msec)', 'Undefined, QRS (msec)',
       'Undefined, QT (msec)', 'Undefined, QTcF (msec)',
       'Undefined, Interpretation', 'Pre dose triplicate 1, Time 24 hrs',
       'Pre dose triplicate 1, HR (bpm)',
       'Pre dose triplicate 1, RR (msec)',
       'Pre dose triplicate 1, PR (msec)',
       'Pre dose triplicate 1, QRS (msec)',
       'Pre dose triplicate 1, QT (msec)',
       'Pre dose triplicate 1, QTcF (msec)',
       'Pre dose triplicate 1, Interpretation',
       'Pre dose triplicate 1, If abnormal, specify',
       'Pre dose triplicate 1, Please specify abnormal interpretation',
       'Pre dose triplicate 2, Time 24 hrs',
       'Pre dose triplicate 2, HR (bpm)',
       'Pre dose triplicate 2, RR (msec)',
       'Pre dose triplicate 2, PR (msec)',
       'Pre dose triplicate 2, QRS (msec)',
       'Pre dose triplicate 2, QT (msec)',
       

In [13]:
triplicate_1 = pru[pru['Campo']=='Pre dose triplicate 1, Time 24 hrs'].Valor.unique()[0]
triplicate_2 = pru[pru['Campo']=='Pre dose triplicate 2, Time 24 hrs'].Valor.unique()[0]
print(triplicate_1)
print(triplicate_2)

08:20
08:25


In [21]:
from datetime import datetime
time_format = '%H:%M'
triplicate_1_formated = datetime.strptime(triplicate_1, time_format)
triplicate_2_formated = datetime.strptime(triplicate_2, time_format)

time_difference = (triplicate_2_formated - triplicate_1_formated).total_seconds() / 60
print(time_difference)


5.0


In [12]:
import math
import numpy as np
prueba_variable = np.nan

if math.isnan(float(prueba_variable)):
    print('Si funciona asi')
else:
    print('No funciona asi') 

Si funciona asi


In [1]:
import math

if math.nan > 4:
    print('Si toca cambiar')
else:
    print('no toca cambiar')

no toca cambiar
